In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf

N = 100
X = np.random.uniform(low=0, high=100, size=N)
Y = 2*X + 1 + np.random.normal(scale=5, size=N)

In [8]:
#Применим метод градиентного спуска
EPOCHS = 20
LEARNING_RATE = 0.0001

In [9]:
costs = []
params = []
preds = []
params = np.random.normal(size=(2,))

for i in range(EPOCHS):
    predictions = params[0] + params[1] * X
    preds.append(predictions)

    cost = np.sum(np.square(predictions - Y)) / (2 * len(predictions))
    costs.append(cost)
    
    params[0] -= LEARNING_RATE * np.sum(predictions - Y) / len(predictions)
    params[1] -= LEARNING_RATE * np.sum((predictions - Y) * X) / len(predictions)  
    print(params[0], params[1])

-0.15459161970370722 1.5845243755008094
-0.1523062643255199 1.7467411651054985
-0.1508997465027756 1.8463674611301508
-0.15003297169060953 1.9075534597415533
-0.14949768310095876 1.9451311373336373
-0.14916597867691517 1.968209634148198
-0.14895930666363022 1.9823833789883079
-0.14882942406690172 1.9910882212098744
-0.1487467021853196 1.996434306676226
-0.14869294438822697 1.999717593126857
-0.1486569751143767 2.001734000167014
-0.1486319308310876 2.0029723462193285
-0.14861359626184417 2.003732841818544
-0.14859938257128533 2.00419986292309
-0.1485876998118703 2.0044866450464247
-0.14857757150853163 2.004662732404708
-0.1485683979537103 2.004770835962494
-0.14855981083369418 2.00483718689462
-0.14855158394658055 2.0048778952220268
-0.14854357836921944 2.004902855003076


In [19]:
# vt = γvt-1 + η∇_θJ(θ), где 
# θ = θ - vθ

γ = 0.9
learning_rate = 0.1

# метод ускоренного градиентного спуска Нестерова в библиотеке TensorFlow:
# tf.compat.v1.train.MomentumOptimizer(
#     learning_rate, momentum, use_locking=False, name='Momentum',
#     use_nesterov=True
# )

# реализация функции ускоренного градиентного спуска Нестерова из модуля TensorFlow:
def _update_nesterov_momentum_numpy(var, accum, g, lr, momentum): # accum - это average_grad (начинается с нуля) 
    var = var + accum * lr * momentum
    accum = accum * momentum + g
    var = var - lr * accum
    var = var - accum * lr * momentum
    return var, accum


# _update_nesterov_momentum_numpy(1.0, 0, 0.1, 0.1, 0.9)


In [ ]:
# метод rmsprop в TensorFlow

tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop', **kwargs
)